# **0. Imports**

In [77]:
import pandas as pd
import numpy as np

## **0.1 Load Dataset**

In [78]:
data = pd.read_csv('../data/data_trab2_econometria.csv')
data.head()

,Data,inflacao,cambio,juros
0,2003,"1,98","3,44","26,38"
1,2003,"2,19","3,59","24,31"
2,2003,"1,14","3,45","23,58"
3,2003,"1,14","3,12","24,90"
4,2003,"0,85","2,95","26,38"


# **1.0 Data Description**

## **1.1 Rename columns**

In [79]:
df1 = data.copy()

In [81]:
cols_new = ['date','cpi', 'exchange_rate','interest_rate']
df1.columns = cols_new

,date,cpi,exchange_rate,interest_rate
0,2003,"1,98","3,44","26,38"
1,2003,"2,19","3,59","24,31"
2,2003,"1,14","3,45","23,58"
3,2003,"1,14","3,12","24,90"
4,2003,"0,85","2,95","26,38"
